Copyright (C) 2022, Intel Corporation

SPDX-License-Identifier: MIT License

# Sequence Classification with BERT in Python using torch-ort inference module:

1. This demo shows how to use Intel® OpenVINO™ integration with Torch-ORT to check grammar in text with ONNX Runtime OpenVINO Execution Provider.

2. We use a sequence classification model textattack/bert-base-uncased-CoLA from HuggingFace models. This model is trained on the BERT architecture to check grammar.

### Import Necessary Resources

In [ ]:
import os
import numpy as np
import time
import pandas as pd
import pathlib

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

import torch
from torch_ort import ORTInferenceModule, OpenVINOProviderOptions

### Preprocess Function
* Use `encode` to:
  * Tokenize the sentence
  * Map tokens to their IDs.
* Pad our input tokens with value 0
* Truncate inuput to MAX_LEN
* Create the attention mask.
* Store the input ids and attention masks for the sentence


In [ ]:
def preprocess_input(tokenizer, sentences):
    # Tokenization & Input Formatting
    # Config: "do_lower_case": true, "model_max_length": 512
    inputs = []

    MAX_LEN = 64

    for sentence in sentences:
        # `encode` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        encoded_sent = tokenizer.encode(
                            sentence,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    )

        # Pad our input tokens with value 0.
        if len(encoded_sent) < MAX_LEN:
            encoded_sent.extend([0]*(MAX_LEN-len(encoded_sent)))

        # Truncate to MAX_LEN
        if len(encoded_sent) > MAX_LEN:
            print("WARNING: During preprocessing, number of tokens for the sentence {}"\
                "exceedeed MAX LENGTH {}. This might impact accuracy of the results".format(
                sentence,
                MAX_LEN
            ))
            encoded_sent = encoded_sent[:MAX_LEN]

        # Create the attention mask.
        #   - If a token ID is 0, then it's padding, set the mask to 0.
        #   - If a token ID is > 0, then it's a real token, set the mask to 1.
        att_mask = [int(token_id > 0) for token_id in encoded_sent]

        # Store the input ids and attention masks for the sentence.
        inputs.append({'input_ids': torch.unsqueeze(torch.tensor(encoded_sent),0),
                'attention_mask': torch.unsqueeze(torch.tensor(att_mask),0)})

    return inputs

### Inference Function
* Runs the inference on the input sentences
* Prints the inference results

In [ ]:
def infer(model, sentences, inputs):
    num_sentences = len(sentences)
    total_infer_time = 0
    results = {}

    # Run inference
    for i in range(num_sentences):
        input_ids = (inputs[i])['input_ids']
        attention_masks = (inputs[i])['attention_mask']
        with torch.no_grad():
            # warm-up
            if i == 0:
               t0 = time.time()
               model(input_ids, attention_masks)
            # infer
            t0 = time.time()
            outputs = model(input_ids, attention_masks)
            t1 = time.time() - t0
            total_infer_time += t1

        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs.logits

        # Move logits
        logits = logits.detach().cpu().numpy()

        # predictions
        pred_flat = np.argmax(logits, axis=1).flatten()
        orig_sent = sentences[i]
        results[orig_sent] = pred_flat[0]

    print("\n Number of sentences: {}".format(num_sentences))
    if num_sentences > 20:
        print(" First 20 results:")
    print("\t Grammar correctness label (0=unacceptable, 1=acceptable)\n")
    count = 0
    for k, v in results.items():
        print("\t{!r} : {!r}".format(k, v))
        if count == 20:
            break
        count = count + 1
    print("\n Average inference time: {:.4f}ms".format((total_infer_time/num_sentences)*1000))
    print(" Total Inference time: {:.4f}ms".format(total_infer_time * 1000))

### Select the inputs

* Use `sentences` to select your inputcsentences to be inferred

* Available backend precisions
  * CPU: FP32
  * GPU(Does not work in collab): FP32, FP16

* Available inference execution providers 
  * OpenVINO

In [ ]:
sentences = ["This is a BERT sample.","User input is valid not."]

backend ="CPU"
precision = "FP32"

### Get and load the model

In [ ]:
# Load Model
# Pretrained model fine-tuned on CoLA dataset from huggingface model hub to predict grammar correctness
model = AutoModelForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-CoLA"
)

# Load Tokenizer & Preprocess input sentences
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-CoLA")
inputs = preprocess_input(tokenizer, sentences)


# Convert model for evaluation
model.eval();

### Run the inference with native PyTorch

In [ ]:
# Infer
infer(model, sentences, inputs)


 Number of sentences: 2
	 Grammar correctness label (0=unacceptable, 1=acceptable)

	'This is a BERT sample.' : 1
	'User input is valid not.' : 0

 Average inference time: 280.6635ms
 Total Inference time: 561.3270ms


### Run the inference with Torch-ORT inference module

In [ ]:
# Select OpenVINO as inference execution provider
if backend and precision:
    provider_options = OpenVINOProviderOptions(backend, precision)
    model = ORTInferenceModule(model, provider_options=provider_options)
else:
    model = ORTInferenceModule(model)

# Infer
infer(model, sentences, inputs)


 Number of sentences: 2
	 Grammar correctness label (0=unacceptable, 1=acceptable)

	'This is a BERT sample.' : 1
	'User input is valid not.' : 0

 Average inference time: 236.1443ms
 Total Inference time: 472.2886ms
